In [1]:
import torch
import alpaca_hotpot_qa
import transformers
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:33: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to

In [2]:
if torch.cuda.is_available():
    device = "cuda"

In [3]:
load_8bit = False
base_model = 'decapoda-research/llama-7b-hf'
lora_weights = 'tloen/alpaca-lora-7b'

tokenizer = LlamaTokenizer.from_pretrained(base_model)
if device == "cuda":
    alp_model = LlamaForCausalLM.from_pretrained(base_model, load_in_8bit=load_8bit,
                    torch_dtype=torch.float16, device_map="auto")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [4]:
#if can't find xxx.json error occurred, check the lora_weights variable, trying using full system path

if device == "cuda":
    alp_model = PeftModel.from_pretrained(alp_model, lora_weights, torch_dtype=torch.float16)

# unwind broken decapoda-research config
alp_model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
alp_model.config.bos_token_id = 1
alp_model.config.eos_token_id = 2

if not load_8bit:
    alp_model.half()  # seems to fix bugs for some users.

alp_model.eval()
if torch.__version__ >= "2" and sys.platform != "win32":
    alp_model = torch.compile(alp_model)

In [5]:
import pickle
#load phonedb data
with open("phone_dataset.pkl", "rb") as f:
    pdb = pickle.load(f)
phonedb_data, name_map = pdb

In [6]:
from alpaca_hotpot_qa import AlpacaHotPotQA

In [7]:
alphot_qa_inference = AlpacaHotPotQA(device, alp_model, tokenizer, phonedb_data, name_map)

In [8]:
alphot_qa_inference.inference("Can you list the battery life for the Apple iPhone 13, Xiaomi Redmi Note 9 Pro,\
and Huawei P30 Pro?")

------------------------------------------------
Step 1: Alpaca extract name tokens

>>>>> Instruction:
 Ignore the input. Extract all phone model names from the input sentence.     Append and prepend '%%%' symbols to each phone model name.

>>>>> Input:
 Can you list the battery life for the Apple iPhone 13, Xiaomi Redmi Note 9 Pro,and Huawei P30 Pro?

Generating ......

<<<<< Output:
 %%Apple iPhone 13%%
%%Xiaomi Redmi Note 9 Pro%%
%%Huawei P30 Pro%%

------------------------------------------------
Using regex to tokenize:
['Apple iPhone 13', 'Xiaomi Redmi Note 9 Pro', 'Huawei P30 Pro']
------------------------------------------------

Step 2: Zero-shot BART classifier extract name keys

Extracted Model Name:  iPhone 13
Extracted Model Name:  Xiaomi Redmi Note 9 Pro
Extracted Model Name:  Huawei P30 Pro
------------------------------------------------


------------------------------------------------
Step 3: Query local database storing scraped text from phonedb.net
Querying local 

100%|██████████| 149/149 [00:01<00:00, 94.78it/s]


Model Name Family:  Xiaomi Redmi Note 9 Pro


100%|██████████| 148/148 [00:01<00:00, 95.81it/s]


Model Name Family:  Huawei P30 Pro


100%|██████████| 139/139 [00:01<00:00, 96.99it/s]



------------------------------------------------
Step 4: Summarizing the text fetched using Alpaca base model
................................................
The summarized context information to be pipe into Alpaca model's prompt (Prompt Engineering)
The Huawei P30 Pro Premium Edition Dual SIM TD-LTE CN VOG-AL00 128GB, Huawei P30 Pro Premium Edition Dual SIM TD-LTE CN VOG-AL10, and Huawei P30 Pro Premium Edition Dual SIM TD-LTE CN VOG-TL00 128GB are all variants of the Huawei P30 Pro. The Huawei P30 Pro Premium Edition Dual SIM TD-LTE CN VOG-AL00 128GB and Huawei P30 Pro Premium Edition Dual SIM TD-LTE CN VOG-AL10 are both full netcom variants of the Huawei P30 Pro, while the Huawei P30 Pro Premium Edition Dual SIM TD-LTE CN VOG-TL00 128GB is a top model of the Huawei P30 Pro. All three variants of the Huawei P30 Pro have 128 GB of storage, 8 GB of RAM, and a 19.5:9 dewdrop AMOLED screen. The Huawei P30 Pro Premium Edition Dual SIM TD-LTE CN VOG-AL00 128GB and Huawei P30 Pro Premium

'The Apple iPhone 13 has a battery life of up to 18 hours. The Xiaomi Redmi Note 9 Pro has a battery life of up to 11 hours. The Huawei P30 Pro has a battery life of up to 19 hours.'

In [9]:
question = '''\
"How do the camera capabilities of the Apple iPhone 12, Samsung Galaxy S21, and Xiaomi Mi 11 compare?"
'''
answer = alphot_qa_inference.inference(question)

------------------------------------------------
Step 1: Alpaca extract name tokens

>>>>> Instruction:
 Ignore the input. Extract all phone model names from the input sentence.     Append and prepend '%%%' symbols to each phone model name.

>>>>> Input:
 "How do the camera capabilities of the Apple iPhone 12, Samsung Galaxy S21, and Xiaomi Mi 11 compare?"


Generating ......

<<<<< Output:
 %%Apple iPhone 12%%,%%Samsung Galaxy S21%%,%%Xiaomi Mi 11%%

------------------------------------------------
Using regex to tokenize:
['Samsung Galaxy S21', 'Xiaomi Mi 11', 'Apple iPhone 12']
------------------------------------------------

Step 2: Zero-shot BART classifier extract name keys

Extracted Model Name:  Samsung Galaxy S21
Extracted Model Name:  Xiaomi Mi 11
Extracted Model Name:  iPhone 12
------------------------------------------------


------------------------------------------------
Step 3: Query local database storing scraped text from phonedb.net
Querying local DataBase ......


100%|██████████| 173/173 [00:01<00:00, 97.19it/s]


Model Name Family:  iPhone 12


100%|██████████| 158/158 [00:01<00:00, 97.52it/s]


Model Name Family:  Xiaomi Mi 11


100%|██████████| 150/150 [00:01<00:00, 95.53it/s]



------------------------------------------------
Step 4: Summarizing the text fetched using Alpaca base model
................................................
The summarized context information to be pipe into Alpaca model's prompt (Prompt Engineering)
Xiaomi Mi 11 Youth 5G Premium Edition Dual SIM TD-LTE CN 256GB M2101K9C is a Chinese variant of Mi11 Lite 5G smartphone with 256 GB UFS 2.2 ROM, 8 GiB LP-DDR4X RAM 802.11ac Wi-Fi, Bluetooth 5.0, NFC, 4G VoLTE, 5G TDD-LTE, 256GB storage, 256GB ROM, 256GB UFS 2.2, 256GB UFS 2.2 ROM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X RAM, 8GB LP-DDR4X R